In [399]:
import numpy as np
import pandas as pd
import os

#this is for loading pages and shit
import bs4
import requests
import re

In [ ]:
def geturls(file):
    line = file.read().replace("\n", " ")
    file.close()
    start = [i for i in range(len(line)) if line.startswith("https", i)]
    end = []
    for index in start:
        i = index
        while line[i] != '"':
            i = i+1
        end.append(i)
    sitelist = []
    for (s,e) in zip(start,end):
        sitelist.append(line[s:e])
    return sitelist

In [ ]:
def getpages(sitelist):
    pages = []
    failed_pages = []
    
    for url in sitelist:
        webpage = requests.get(url)
        statcode = str(webpage.status_code)
        # status code starting with a 2 generally indicates success,
        # and a code starting with a 4 or a 5 indicates an error.
        # page.content
        soup = bs4.BeautifulSoup(webpage.content,'html.parser')
        page = str(soup)
        
        readstart = 0
        readend = 0
        for i in range(len(page)):
            if page.startswith(".entry-header", i):
                readstart = i
            if page.startswith("Explanation:", i):
                readend = i
        while (not page.startswith("</p>", readstart)) and readend < len(page): readstart = readstart + 1
        while (not page.startswith("</p>", readend)) and readend < len(page): readend = readend + 1
        page = page[readstart:readend]
        
        if readstart == 0 or statcode.startswith('4') or statcode.startswith('5'):
            failed_pages.append(url)
        else:
            pages.append(page)
    return (pages, failed_pages)

In [336]:
def getdirtyquestions(pages):
    dirtyquestions = []
    
    for page in pages:
        page = page.replace("<br/>"," ")
        qno = 1
        for i in range(len(page)):
            if page.startswith("<p>"+str(qno)+".", i):
                qno = qno+1
                j = i
                while (not page.startswith("Explanation:", j)) and j < len(page): j = j + 1
                while (not page.startswith("</div>", j)) and j < len(page): j = j + 1
                dirtyquestions.append(page[i+3:j])
    return dirtyquestions

In [366]:
def getcleanquestions(dirtyquestions):
    cleanquestions = []
    for q in dirtyquestions:
        for i in range(len(q)): 
            if q.startswith("<span", i): break
        for j in range(len(q)): 
            if q.startswith("Answer:", j): break
        cleanquestions.append(q[0:i] + q[j:])
    return cleanquestions
            

In [414]:
os.chdir("/kaggle/input")
file = open("SANCRALWER.txt")
sitelist = geturls(file)

# one webpage had no questions
sitelist = sitelist[0:95]

# fetching the relevant parts of each webpage
(pages, failed_pages) = getpages(sitelist)
print(pages[0])

</p>
<p>1. IBM and ________ have announced a major initiative to use Hadoop to support university courses in distributed computer programming.<br/>
a) Google Latitude<br/>
b) Android (operating system)<br/>
c) Google Variations<br/>
d) Google<br/>
<span class="collapseomatic" id="id5ede214b2badb" tabindex="" title="View Answer">View Answer</span><div class="collapseomatic_content" id="target-id5ede214b2badb">Answer: d<br/>
Explanation: Google and IBM Announce University Initiative to Address Internet-Scale.</div>
<div class="mobile-incontent-ads">
<div style="font-size:13px; text-align: center;">advertisement</div>
<div class="_ap_apex_ad" id="ba924445-0f94-4f60-acc3-655f343484d9">
<script>
	    var adpushup = adpushup || {};
	    adpushup.que = adpushup.que || [];
	    adpushup.que.push(function() {
		adpushup.triggerAd("ba924445-0f94-4f60-acc3-655f343484d9");
	    });
	</script>
</div>
</div>
<p>2. Point out the correct statement.<br/>
a) Hadoop is an ideal environment for extracting

In [415]:
# getting questions from relevant sections of HTML webpage
dirtyquestions = getdirtyquestions(pages)
print(dirtyquestions[0])

1. IBM and ________ have announced a major initiative to use Hadoop to support university courses in distributed computer programming. 
a) Google Latitude 
b) Android (operating system) 
c) Google Variations 
d) Google 
<span class="collapseomatic" id="id5ede214b2badb" tabindex="" title="View Answer">View Answer</span><div class="collapseomatic_content" id="target-id5ede214b2badb">Answer: d 
Explanation: Google and IBM Announce University Initiative to Address Internet-Scale.


In [411]:
# this was for testing whether standard format was being followed
separator = " "
testpages = separator.join(pages)
print(testpages.count("Answer:"))

958


In [380]:
countspan = 0
countnospan = 0
indexwithoutspan = 0
for qi in range(len(dirtyquestions)):
    if dirtyquestions[qi].count("<span") == 0:
        countnospan = countnospan + 1
        indexwithoutspan = qi
    else:
        countspan = countspan + 1
print("Span : " + str(countspan) + "\nNo Span : " + str(countnospan) + "\nIndex Without Span : " + str(indexwithoutspan))
print("\nQuestion without span : \n" + dirtyquestions[indexwithoutspan])

# there were three questions deviating from the standard HTML format, which were treated seperately as no span, owing to their missing <span> tags
nospanquestion = dirtyquestions[375]

# removing no span question from dirtyquestions
dirtyquestions = dirtyquestions[0:375] + dirtyquestions[376:956]

Span : 955
No Span : 1
Index Without Span : 375

Question without span : 
8. Which is the additional command line option is available in Hive 0.10.0? 
a) –database &lt;dbname&gt; 
b) –db &lt;dbname&gt; 
c) –dbase &lt;&lt;dbname&gt;
d) All of the mentioned
[expand title="View Answer"]Answer: a
Explanation: Database is specified which is to be used.[/expand] 

9. The CLI when invoked without the -i option will attempt to load $HIVE_HOME/bin/.hiverc and $HOME/.hiverc as _______ files.
a) processing
b) termination
c) initialization
d) none of the mentioned
[expand title="View Answer"]Answer: c 
Explanation: Hiverc file is loaded as per options selected.[/expand] 

10. When $HIVE_HOME/bin/hive is run without either the -e or -f option, it enters _______ mode.
a) Batch
b) Interactive shell
c) Multiple
d) None of the mentioned
[expand title="View Answer"]Answer: b
Explanation: Use ";" (semicolon) to terminate commands for multiple options available.[/expand] 

<strong>Sanfoundry Global Educat

In [412]:
# cleaning no span question manually
nospanquestion = "8. Which is the additional command line option is available in Hive 0.10.0? \na) –database &lt;dbname&gt; \nb) –db &lt;dbname&gt; \nc) –dbase &lt;&lt;dbname&gt;\nd) All of the mentioned\nAnswer: a\nExplanation: Database is specified which is to be used.\n\n9. The CLI when invoked without the -i option will attempt to load $HIVE_HOME/bin/.hiverc and $HOME/.hiverc as _______ files.\na) processing\nb) termination\nc) initialization\nd) none of the mentioned\nAnswer: c \nExplanation: Hiverc file is loaded as per options selected.\n\n10. When $HIVE_HOME/bin/hive is run without either the -e or -f option, it enters _______ mode.\na) Batch\nb) Interactive shell\nc) Multiple\nd) None of the mentioned\nAnswer: b\nExplanation: Use \";\" (semicolon) to terminate commands for multiple options available."

In [416]:
# cleaning span questions
cleanquestions = getcleanquestions(dirtyquestions)
print(cleanquestions[0])

1. IBM and ________ have announced a major initiative to use Hadoop to support university courses in distributed computer programming. 
a) Google Latitude 
b) Android (operating system) 
c) Google Variations 
d) Google 
Answer: d 
Explanation: Google and IBM Announce University Initiative to Address Internet-Scale.


In [418]:
# putting everything together
final = "\n\n".join(cleanquestions)
final = final + "\n\n" + nospanquestion

In [419]:
os.chdir("/kaggle/working")
with open("test.txt", "w") as f:
    f.write(final)

OSError: [Errno 30] Read-only file system: 'test.txt'